# Detect (with the) Model
## Step 1 - run the TFLite model locally

In this notebook, we are replicating  some of the original MobileNet TensorFlow Lite functionality.  That means taking the TensorFlow model (checkpoint) and converting it to a TensorFlow Lite model.   This is not quite what the SageMaker tutorial does.   The tutorial sticks with TensorFlow.   Confirm that the Lite model works first - because that is a known path then:
- Detect Step 2 - run endpoint as a TensorFlow model
- Detect Step 3 - run endpoint as a TensorFlow Lite Model -- the advantage of a Lite model is less latency, faster, less resources -- with slightly less accuracy.    Lite models are the path towards IoT deployments and TPU compatible models

We are testing the model before we deploy it as an endpoint.   This is optional but definitely makes sense as we learn this process

In [1]:
import os

## Global Environment Variables

We are using objects and scripts in the project as much as possible.   They require environment variables to pass along where stuff is stored

### Note 
This notebook assumes output_tflite_graph.tflite is already on the SageMaker server (it will be if you ran the Train Step 3 notebook.)  If not, you can get it from a SageMaker Training Job /output/tflite_model on S3

In [8]:
PROJECT_DIR = os.getcwd()
IMAGE_DIR = os.path.join(PROJECT_DIR, "data/new_jpeg_images")
MODEL_NAME = 'tf_lite'   # not using the edge_tpu 
MODEL_PATH = os.path.join(PROJECT_DIR, "tflite_model/output_tflite_graph.tflite")
LABEL_MAP = os.path.join(PROJECT_DIR, "code/cfa_prod_label_map.pbtxt")
DISPLAY = "None"
ANNOTATION_DIR = os.path.join(PROJECT_DIR, "data/unverified_annotations")
S3_TEST_IMAGES = "s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/"

## Data
Copy the test images locally

### aws2
- if local, remember to update our credentials
- if using AWS CLI Version2,   
https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-linux-mac.html#cliv2-linux-mac-install  
- remember, it's:  
`aws2 s3 cp`

In [9]:
# version 2
! aws2 s3 cp {S3_TEST_IMAGES} {IMAGE_DIR} --recursive

download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562780998.jpg to data/new_jpeg_images/20190710_variety_1562780998.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781004.jpg to data/new_jpeg_images/20190710_variety_1562781004.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781005.jpg to data/new_jpeg_images/20190710_variety_1562781005.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562780997.jpg to data/new_jpeg_images/20190710_variety_1562780997.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781003.jpg to data/new_jpeg_images/20190710_variety_1562781003.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781001.jpg to data/new_jpeg_images/20190710_variety_1562781001.jp

download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781052.jpg to data/new_jpeg_images/20190710_variety_1562781052.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781059.jpg to data/new_jpeg_images/20190710_variety_1562781059.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781036.jpg to data/new_jpeg_images/20190710_variety_1562781036.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781035.jpg to data/new_jpeg_images/20190710_variety_1562781035.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781053.jpg to data/new_jpeg_images/20190710_variety_1562781053.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781034.jpg to data/new_jpeg_images/20190710_variety_1562781034.jp

download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781109.jpg to data/new_jpeg_images/20190710_variety_1562781109.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781110.jpg to data/new_jpeg_images/20190710_variety_1562781110.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781086.jpg to data/new_jpeg_images/20190710_variety_1562781086.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781097.jpg to data/new_jpeg_images/20190710_variety_1562781097.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781105.jpg to data/new_jpeg_images/20190710_variety_1562781105.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/20190710_variety_1562781111.jpg to data/new_jpeg_images/20190710_variety_1562781111.jp

## Detect
The detect.py script will read a directory of images, infer and generate XML (unverified) Annotations (VOC PASCAL schema).    Unverified means if you run them through a labeling program like labelImg, they do not have the verified attribute.    (Downstream, they will be ignored if you try to fold them into training data.)

In [ ]:
! python code/detect.py --image_dir {IMAGE_DIR} --model_name {MODEL_NAME} --model_path {MODEL_PATH} --label_map_path {LABEL_MAP} --display {DISPLAY} --annotation_dir {ANNOTATION_DIR}

In [ ]:
# tarball the annotations
os.chdir("data")
! tar -czvf  unverif_annotations.tar.gz unverified_annotations

## Conclusion

If you want the annotations - you'll find the annotations tarball in data/ directory
use the Notebook browser to download it

The fastest, easiest way to review (and correct / verify) is to use labelImg program which will merge the image and annotation

The main conclusion here is our model works.